In [1]:
from new_helper import *

# DIRECTORY_PATH = "upload.old/"

# upload_documents_in_directory(DIRECTORY_PATH)


# Example usage:
directory_path = "upload"
process_files(directory_path)


NameError: name 'tika' is not defined

In [ ]:
from Crypto.Hash import keccak
import os
import binascii
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import sys
import json
from ipfs_functions import *
import logging

if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]

iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))

# Function to link details using blockchain
def set_account_detail(address, account, key, value):
    params = integration_helpers.get_first_four_bytes_of_keccak(
        b"setAccountDetail(string,string,string)"
    )
    no_of_param = 3
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(account)  # account id
    params = params + integration_helpers.argument_encoding(key)  # key
    params = params + integration_helpers.argument_encoding(value)  # value
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash


logging.basicConfig(level=logging.INFO)
sh = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
sh.setFormatter(formatter)
logging.getLogger().addHandler(sh)

def upload_documents_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        logging.info(f"Processing file: {filename}")

        if not os.path.basename(filename).startswith('.'):
            file_path = os.path.join(directory_path, filename)

            try:
                # Upload the file to IPFS and get its CID
                file_cid = upload_file_to_ipfs(file_path)
                logging.info(
                    f"File {filename} uploaded to IPFS with CID: {file_cid}"
                )

                # # Call some other function to handle metadata_cid and upload it to IPFS
                # metadata_cid = handle_metadata_and_upload_to_ipfs(file_path)

                # # Add the file_cid and metadata_cid into the database or do something else
                # add_file_cids_to_database(file_cid, metadata_cid)
            except Exception as e:
                logging.error(f"Error processing file {filename}: {e}")



DIRECTORY_PATH = "upload.old/"

upload_documents_in_directory(DIRECTORY_PATH)

